In [1]:
# Import Packages
import pandas as pd
import numpy as np

# Center For Community Service


## Part I: Data Cleaning

##### CSV File 1: 4.29.2023_Survey_Results.csv

In [3]:
# Read survey data from the csv file
df = pd.read_csv("4.29.2023_Survey_Results.csv")

In [4]:
# Keep numerical and binary columns
quant_df = df.drop("Who took this survey (initials)", axis=1)
quant_df.drop("Any additional comments?", axis=1, inplace=True)
quant_df.drop("Who's survey is this?", axis=1, inplace=True)
quant_df.drop("Timestamp", axis=1, inplace=True)
quant_df.drop("Unnamed: 16", axis=1, inplace=True)

# Change headers
quant_df.columns = ['bike_rider', 'zipcode', 'ebike_rider', 'commute_purpose',
                   'miles', 'try_bike', 'ebike_class', 'private_locker', 'group_locker',
                   'ebike_rules', 'safty', 'interested']

In [5]:
# convert zipcode values to string
quant_df['zipcode'] = quant_df['zipcode'].apply(lambda x: str(x).replace('.0', ''))

# convert all letters to lowercase
columns = quant_df.columns
for column in columns:
    quant_df[column] = quant_df[column].str.lower()

In [6]:
# Set cells to null if its unit is time
quant_df['miles'] = quant_df['miles'].apply(lambda x: np.nan if pd.isnull(x) or 'min' in x else x)
quant_df['miles'] = quant_df['miles'].str.extract(r'(\d+)', expand=False).astype(float).astype(pd.Int64Dtype())

##### CSV File 2: 6.08.2023_Online_Survey_Results.csv

In [7]:
df2= pd.read_csv("6.08.2023_Online_Survey_Results.csv")

In [8]:
# Keep question columns only
df2 = df2[['Q1', 'Q2', 'Q3',
           'Q4_1', 'Q4_2', 'Q4_3', 'Q4_4', 
           'Q4_5', 'Q4_6', 'Q4_7', 'Q5', 'Q6',
           'Q7', 'Q8', 'Q9', 'Q10', 'Q11']]

# Drop row 1
df2 = df2.drop(1)

In [9]:
# Change column names
df2.columns = ['bike_rider', 'zipcode', 'ebike_rider', 'commute_purpose_1', 'commute_purpose_2',
               'commute_purpose_3', 'commute_purpose_4', 'commute_purpose_5', 'commute_purpose_6',
               'commute_purpose_7', 'miles', 'try_bike', 'ebike_class', 'locker',
               'ebike_rules', 'safty', 'interested']
df2 = df2.drop(0)

In [10]:
# Set cells to null if its unit is time
df2['miles'] = df2['miles'].apply(lambda x: np.nan if pd.isnull(x) or 'min' in x else x)
df2['miles'] = df2['miles'].str.extract(r'(\d+)', expand=False).astype(float).astype(pd.Int64Dtype())

In [11]:
# Keep U.S. zipcodes only
df2['zipcode'] = df2['zipcode'].apply(lambda x: np.nan if pd.isnull(x) or len(x) != 5 else x)

##### Merge quant_df Dataframe and df2 Dataframe

In [12]:
# Combine two dataframes
df = pd.concat([quant_df, df2], ignore_index=True)

# Convert all letters to lowercase
columns = df.columns
for column in columns:
    if column != 'zipcode' and column != 'miles':
        df[column] = df[column].str.lower()

# Drop the column for 'combination' among communte_purpose
df = df.drop('commute_purpose_6', axis=1)

In [13]:
# Convert all columns except for 'miles' and 'zipcode' to string
columns = df.columns
for column in columns:
    if column != 'zipcode' and column != 'miles':
        df[column] = df[column].astype(str)

# Rename each commute purpose columns
df = df.rename(columns={'commute_purpose_1': 'by_public', 
                        'commute_purpose_2': 'by_car',
                        'commute_purpose_3': 'by_bike',
                        'commute_purpose_4': 'by_ebike',
                        'commute_purpose_5': 'by_walking',
                        'commute_purpose_7': 'no_commute'})

In [14]:
# Transfer information in 'commute_purpose' column in to 'by_' columns
df['by_public'] = df.apply(lambda row: 1 if 'public' in row['commute_purpose']
                                                          or 'trolley' in row['commute_purpose'] 
                                                          or 'mts' in row['commute_purpose'] 
                                                          or 'bus' in row['commute_purpose']
                                          else row['by_public'], axis=1)

df['by_car'] = df.apply(lambda row: 1 if 'car' in row['commute_purpose'] or 'uber' in row['commute_purpose'] 
                                      else row['by_car'], axis=1)
df['by_bike'] = df.apply(lambda row: 1 if 'bike' in row['commute_purpose'] and 'e bike' not in row['commute_purpose']
                                       else row['by_bike'], axis=1)
df['by_ebike'] = df.apply(lambda row: 1 if 'e bike' in row['commute_purpose']
                                       else row['by_ebike'], axis=1)
df['by_walking'] = df.apply(lambda row: 1 if 'walk' in row['commute_purpose']
                                       else row['by_walking'], axis=1)
df['no_commute'] = df.apply(lambda row: 1 if 'i do not commute' in row['commute_purpose']
                                       else row['no_commute'], axis=1)

# Convert value in 'by_' columns to 1 if the cell != nan
df['by_public'] = df['by_public'].apply(lambda x: 1 if x != 'nan' else 0)
df['by_car'] = df['by_car'].apply(lambda x: 1 if x != 'nan' else 0)
df['by_bike'] = df['by_bike'].apply(lambda x: 1 if x != 'nan' else 0)
df['by_ebike'] = df['by_ebike'].apply(lambda x: 1 if x != 'nan' else 0)
df['by_walking'] = df['by_walking'].apply(lambda x: 1 if x != 'nan' else 0)
df['no_commute'] = df['no_commute'].apply(lambda x: 1 if x != 'nan' else 0)

# Drop 'commute_purpose' column since it's been one-hot encoded into 'by_' columns
df = df.drop('commute_purpose', axis=1)

In [15]:
# Keep answersin the 'ebike_rider' column to either yes or no
df['ebike_rider'] = df['ebike_rider'].apply(lambda x: 'yes' if x != 'nan' and x != 'no' else x)

In [16]:
# Reorder the columns to make it align with the order of the survey questions
df = df[['bike_rider', 'zipcode', 'ebike_rider',
         'by_public', 'by_car', 'by_bike', 'by_ebike', 'by_walking', 'no_commute',
         'miles', 'try_bike', 'ebike_class', 'private_locker', 'group_locker', 'locker',
         'ebike_rules', 'safty', 'interested']]

In [17]:
# Save df as quant_df, which is the name of the df that will be used in the analysis
quant_df = df

##### CSV File 3: June_10_2023_Survey_Data.csv

In [18]:
df3 = pd.read_csv('June_10_2023_Survey_Data.csv')

In [19]:
concatenated_df = pd.concat([quant_df, df3], sort=False)

# Reset the index of the concatenated dataframe
concatenated_df = concatenated_df.reindex(columns=df.columns)

In [20]:
def replace_with_yes(row):
    if type(row) != float:
        if 'yes' in row:
            return 'yes'
    return row

def replace_with_no(row):
    if type(row) != float:
        if 'no' in row or 'but' in row:
            return 'no'
    return row

def replace_with_maybe(row):
    if type(row) != float:
        if 'may' in row:
            return 'maybe'
    return row

def replace_with_dn(row):
    if type(row) != float:
        if "don't know" in row or 'don’t know' in row:
            return 'don\'t know'
    return row

def replace_with_nan(row):
    if type(row) != float:
        if 'nan' in row:
            return np.nan
    return row

def replace_with_depends(row):
    if type(row) != float:
        if 'depends' in row:
            return 'somewhat'
    return row

In [21]:
concatenated_df['bike_rider'] = concatenated_df['bike_rider'].apply(replace_with_yes)
concatenated_df['bike_rider'] =concatenated_df['bike_rider'].apply(replace_with_no)

In [22]:
concatenated_df['ebike_rules'] = concatenated_df['ebike_rules'].apply(replace_with_dn)

In [23]:
# convert zipcode values to string
concatenated_df['zipcode'] = concatenated_df['zipcode'].apply(lambda x: str(x).replace('.0', ''))

In [24]:
concatenated_df['try_bike'] = concatenated_df['try_bike'].apply(replace_with_nan)
concatenated_df['private_locker'] = concatenated_df['private_locker'].apply(replace_with_nan)
concatenated_df['group_locker'] = concatenated_df['group_locker'].apply(replace_with_nan)
concatenated_df['group_locker'] = concatenated_df['group_locker'].apply(replace_with_depends)
concatenated_df['locker'] = concatenated_df['locker'].apply(replace_with_nan)
concatenated_df['safty'] = concatenated_df['safty'].apply(replace_with_nan)
concatenated_df['interested'] = concatenated_df['interested'].apply(replace_with_nan)
concatenated_df['interested'] = concatenated_df['interested'].apply(replace_with_no)
concatenated_df['interested'] = concatenated_df['interested'].apply(replace_with_yes)
concatenated_df['interested'] = concatenated_df['interested'].apply(replace_with_maybe)

In [25]:
concatenated_df

,bike_rider,zipcode,ebike_rider,by_public,by_car,by_bike,by_ebike,by_walking,no_commute,miles,try_bike,ebike_class,private_locker,group_locker,locker,ebike_rules,safty,interested
0,no,92115,no,0,0,0.0,0,0,1,<NA>,maybe,no,very,very,NaN,don't know,not very,no
1,no,92129,no,0,1,0.0,0,0,0,<NA>,no,a bit,NaN,NaN,NaN,not very,somewhat,no
2,no,92115,yes,1,0,0.0,0,0,0,1,maybe,a bit,very,somewhat,NaN,don't know,very,yes
3,no,92562,no,0,0,0.0,0,0,1,<NA>,NaN,no,NaN,NaN,NaN,don't know,not concerned,maybe
4,yes,92109,no,0,0,0.0,0,0,1,<NA>,NaN,no,NaN,NaN,NaN,don't know,not concerned,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,no,91945,no,0,1,NaN,0,0,0,20,NaN,a bit,NaN,NaN,somewhat,not very,NaN,yes
37,yes,92108,no,0,1,NaN,0,0,0,10,NaN,no,NaN,NaN,somewhat,not very,NaN,yes
38,yes,92117,no,0,1,NaN,0,0,0,15,NaN,no,NaN,NaN,very,not very,NaN,maybe
39,yes,92102,no,0,1,NaN,0,0,0,12,NaN,no,NaN,NaN,somewhat,somewhat,NaN,maybe


Here's a description of each feature (column) in the quant_df DataFrame:

Header | Description
-------|------------
bike_rider | Are you a bike rider?
zipcode | What is your zip code?
ebike_rider | Do you or anyone in your family ride an e-bike?
by_public | How do you commute - to work, school, or for other purposes? - Answered by public transportaion
by_car | How do you commute - to work, school, or for other purposes? - Answered by car
by_bike | How do you commute - to work, school, or for other purposes? - Answered by bike
by_ebike | How do you commute - to work, school, or for other purposes? - Answered by e bike
by_walking | How do you commute - to work, school, or for other purposes? - Answered by walking
no_commute | How do you commute - to work, school, or for other purposes? - Answered no need for commute
miles | How far (ROUND TRIP) do you commute for any purpose each day?
try_bike | Would you commute by e-bike if given the opportunity?
ebike_class | Do you know the differences between Class 1, Class 1, and Class 3 e-bikes?
private_locker | How comfortable would you be storing your bike in a private bike locker while you are commuting?
group_locker | How comfortable would you be storing your bike in a group bike locker while you are commuting?
locker | How comfortable would you be storing your bike in a bike locker while you are commuting?
ebike_rules | How familiar are you and those in your family with the rules of the road for e-bikes?
safty | How concerned are you about e-bikes and e-bike safety in San Diego County?
interested | Are you interested in learning more about e-bikes?


#### Export Data Frame as Excel

In [ ]:
# Export DataFrame to Excel
excel_filename = 'cce_survey_data.xlsx'
concatenated_df.to_excel(excel_filename, index=False)